In [4]:
import pandas as pd
import numpy as np
import json
import urllib
import requests

Class Roster

In [ ]:
def getFineData(row): return row['enrollGroups'][0]['classSections'][0]['meetings']
def getProf(row): 
  x = lambda r: r[0]['instructors'][0]['firstName'].strip() + ' '+r[0]['instructors'][0]['lastName'].strip()
  try: 
    return x(row)
  except: return ""
def getData(fineData, obj): 
  try:
    x = lambda r: r[0][obj]
    return x(fineData)
  except: return ""

In [ ]:
def scrape_class_roster(sem, csv_name):
  subjects_url = 'https://classes.cornell.edu/api/2.0/config/subjects.json?roster='+sem
  subjects_json = json.loads(urllib.request.urlopen(subjects_url).read())
  subject_list = [x['value'] for x in subjects_json['data']['subjects']]
  courses = pd.DataFrame(columns = ["Dept", "Number", "Course_Name", "Professor","Start_Time", "End_Time", "Location","Description","Pre/coreqs","Offered"])

  for subject in subject_list: 
    url = "https://classes.cornell.edu/api/2.0/search/classes.json?roster="+sem+f"&subject={subject}"
    data_json = json.loads(urllib.request.urlopen(url).read())
    df = pd.DataFrame(data_json['data']['classes'])

    df['prof'] = df.apply(lambda row: getProf( getFineData(row)), axis=1)
    df['startTime'] = df.apply(lambda row: getData(getFineData(row), 'timeStart'), axis=1)  
    df['endTime'] = df.apply(lambda row: getData(getFineData(row), 'timeEnd'), axis=1)  
    df['location'] = df.apply(lambda row: getData(getFineData(row), 'facilityDescr'), axis=1)  

    df = df[["subject", "catalogNbr", "titleShort", "prof", "startTime", "endTime", "location", "description", "catalogPrereqCoreq", "catalogWhenOffered"]]
    df = df.rename(columns={"subject": "Dept", 'catalogNbr':"Number", "titleShort":"Course_Name", "prof":"Professor", "startTime":"Start_Time", "endTime":"End_Time", "location": "Location", "description":"Description", "catalogPrereqCoreq":"Pre/coreqs", "catalogWhenOffered":"Offered"})

    courses = courses.merge(df, how='outer') 

  courses.to_csv('data/class_roster_'+sem+'.csv')


In [114]:
sem = "SP22"
scrape_class_roster(sem)

In [170]:
sem = "FA22"
scrape_class_roster(sem)

CUreviews

In [21]:
def scrape_cureviews (courses):
  url = 'https://www.cureviews.org/v2/getCourseByInfo/'
  courses['Dept + Number'] = courses.Dept + " " + courses.Number.map(str)

  class_diff = {}
  rating = {}
  workload = {}

  for index, row in courses['Dept + Number'].iteritems():
    try:
      jsn = {'number':int(row.split(' ')[1]), 'subject':row.split(' ')[0].lower()}
      # print(jsn)
      result = requests.post(url, json=jsn).json()['result']
      class_diff[row] = result['classDifficulty']
      rating[row] = result['classRating']
      workload[row] = result['classWorkload']
      
    except:
      class_diff[row] = ""
      rating[row] = ""
      workload[row] = ""
  
  courses["class_diff"] = courses['Dept + Number'].map(class_diff)
  courses["rating"] = courses['Dept + Number'].map(rating)
  courses["workload"] = courses['Dept + Number'].map(workload)

  return courses

In [22]:
courses_FA = scrape_cureviews(pd.read_csv('data/class_roster_FA22.csv'))
courses_SP = scrape_cureviews(pd.read_csv('data/class_roster_SP22.csv'))

In [23]:
courses_FA.to_csv('data/CUReviews_FA22')
courses_SP.to_csv('data/CUReviews_SP22')

In [24]:
courses = courses_FA.merge(courses_SP, how='outer')

In [25]:
courses.head()

,Unnamed: 0,Dept,Number,Course_Name,Professor,Start_Time,End_Time,Location,Description,Pre/coreqs,Offered,Dept + Number,class_diff,rating,workload
0,0,AAS,2100,South Asian Diaspora,Viranjini Munasinghe,02:55PM,04:10PM,Malott Hall 230,This interdisciplinary course (with an emphasi...,NaN,Fall.,AAS 2100,None,None,None
1,1,AAS,2130,Intro To Asian American Hist,Derek Chang,11:25AM,12:40PM,Goldwin Smith Hall G64-Kau Aud,"An introductory history of Chinese, Japanese, ...",NaN,Fall.,AAS 2130,1.666667,4,2.333333
2,2,AAS,3885,Race and War in History,Tejasvi Nagaraja,01:00PM,02:15PM,Ives Hall 219,"Across twentieth-century history, race and war...",NaN,Fall.,AAS 3885,,,
3,3,AAS,4040,Fictions of Dictatorship,Christine Balance,02:40PM,05:10PM,Malott Hall 205,"Fictions of dictatorship, as termed by scholar...",NaN,Fall.,AAS 4040,None,None,None
4,4,AAS,4950,Independent Study,Viranjini Munasinghe,NaN,NaN,NaN,Independent reading course in topics not cover...,NaN,"Fall, Spring.",AAS 4950,None,None,None


Rate My Prof

In [2]:
# from https://github.com/tisuela/ratemyprof-api/tree/master/ratemyprof_api
from professor import *
from ratemyprof_api import *

In [3]:
rmp = RateMyProfApi(298).scrape_professors()

In [6]:
num_ratings = {}
prof_rat = {}
prof_diff = {}
# count = 0
for prof in rmp:
    
    try:
        result = urllib.requests.get("https://www.ratemyprofessors.com/ShowRatings.jsp?tid=" + str(prof))
        page = result.content.decode('utf-8')

        num_ratings[rmp[prof].name] = rmp[prof].num_of_ratings
        prof_rat[rmp[prof].name] = rmp[prof].overall_rating
        diff = page.index('avgDifficulty')
        prof_diff[rmp[prof].name] = page[page.index(':', diff + 1)+1:page.index(',', diff + 1)]
        
    except:
        print(rmp[prof].name)
    # count += 1
    # if count > 10:
        # assert False

Aditya Manthri


In [7]:
profs = []
for prof in rmp:
    profs.append(rmp[prof].name)
prof_info = pd.DataFrame(columns = ['Professor', 'num_ratings', 'prof_rat', 'prof_diff'])
prof_info['Professor'] = profs

prof_info['num_ratings'] = prof_info['Professor'].map(num_ratings)
prof_info['prof_rat'] = prof_info['Professor'].map(prof_rat)
prof_info['prof_diff'] = prof_info['Professor'].map(prof_diff)
prof_info.to_csv('data/rmp.csv')

In [26]:
# prof_info = pd.read_csv('data/rmp.csv')

Concatenating medians

In [27]:
courses_w_prof = courses.merge(prof_info, on="Professor", how="outer")

In [28]:
medians = pd.read_csv('data/CornellMedianGrades.csv')
medians = medians[1:]

course_medians = courses_w_prof.merge(medians, on="Dept + Number", how="outer")

In [29]:
course_medians.to_csv('data/merged_data.csv')